In [1]:
#importing required libraries
 
import os
import pandas as pd
import numpy as np

from typing import List, Dict

In [2]:
#importing functions to calculate TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
#importing the dataset
songs = pd.read_csv('/Users/anjalirai/Downloads/songdata.csv')


In [6]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)


In [8]:
songs['text'] = songs['text'].str.replace(r'\n', '')


<ipython-input-8-215fe46be90d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [9]:
#using tf-idf to calculate the frequency score of the words of the lyrics
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')


In [10]:
#creating matrix of the tfidf scores
lyrics_matrix=tfidf.fit_transform(songs['text'])

In [11]:
#calculating similarities between lyrics using cosine distance
cosine_similarities = cosine_similarity(lyrics_matrix) 


In [12]:
similarities = {}


In [13]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]


In [14]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [16]:
recommedations = ContentBasedRecommender(similarities)


In [17]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [18]:
recommedations.recommend(recommendation)


The 4 recommended songs for My Generation are:
Number 1:
My Girl by Phil Collins with 0.298 similarity score
--------------------
Number 2:
Jive Talkin' by Chaka Khan with 0.265 similarity score
--------------------
Number 3:
Bout It by Yung Joc with 0.261 similarity score
--------------------
Number 4:
Give Peace A Chance by Cyndi Lauper with 0.247 similarity score
--------------------
